# Amazon Alexa Review Sentiment Analysis: pos, neu, neg

In [ ]:
# add gitinore 
# download and extract data from kaggle or json file..
# don't track json, zip, csv, DS_store file

In [1]:
# installing the kaggle library to fetch the dataset from kaggle 

%pip install kaggle 

# configuring 
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle / 
!chmod 600 ~/.kaggle/kaggle.json

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
cp: kaggle.json: No such file or directory
cp: /Users/rt/.kaggle is a directory (not copied).


In [ ]:
# API to fetch the dataset from kaggle:
! kaggle datasets download -d sid321axn/amazon-alexa-reviews

In [ ]:
# extracting the dataset we just downloaded
from zipfile import ZipFile
dataset_path = "./amazon-alexa-reviews.zip"
with ZipFile(dataset_path, "r") as zip:
    zip.extractall(".")
    print("Dataset extracted successfully")